In [1]:
import papermill as pm
import scrapbook as sb
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import os

%load_ext sql

/Users/jin/opt/anaconda3/envs/lectures/lib/python3.9/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


We are instructed to run over all tokens in the database for the last 2 complete weeks in the database. (Note, I interpret the expression "last 2 complete weeks" as the most recent 14 days observed in the database). Therefore we need some preliminary skimming of the database for such conditions. 

In [ ]:
conn = sqlite3.connect('data/data.db')
cur = conn.cursor()

In [7]:
%%sql sqlite:///data/data.db

SELECT 
    token
FROM ohlc 
GROUP BY 1

Done.


token
AAVE
ADA
ATOM
AVAX
BTC
COMP
CRV
DOT
ETH
SOL


In [17]:
%%sql tokens <<

SELECT 
    token
FROM ohlc 
GROUP BY 1

 * sqlite:///data/data.db
Done.
Returning data to local variable tokens


We can see that there are in total such tokens above to consider. We can retrieve each token name directly from the **tokens** object so we leave it be in sql form.

In [20]:
%%sql

SELECT 
    ts
FROM ohlc 
GROUP BY 1
ORDER BY 1 DESC
LIMIT 5

 * sqlite:///data/data.db
Done.


ts
2021-12-15 00:00:00
2021-12-14 23:00:00
2021-12-14 22:00:00
2021-12-14 21:00:00
2021-12-14 20:00:00


The last recorded time is **2021-12-15 00:00:00**. Therefore, we can observe (plot and calculate) from **2021-12-01 00:00:00**. I run the **my_template.ipynb** file with the subsequent cells for each token over the most recent 2 weeks.

In [38]:
run_dates = [datetime(2021, 12, 1)]
connection_string = 'sqlite:///data/data.db'

In [39]:
run_dates

[datetime.datetime(2021, 12, 1, 0, 0)]

In [40]:
base_output_folder = os.path.expanduser('my_outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [44]:
for run_date in run_dates:
    for token in tokens:
        print(f'running for {token[0]} for date {run_date} to {run_date + timedelta(days=14)}', end='\r')
        res = pm.execute_notebook(
            'my_template.ipynb',
            f'{base_output_folder}/my_analysis_{token[0]}_{run_date}.ipynb',
            parameters = {
                'from_date': f'{run_date}',
                'to_date': f'{run_date + timedelta(days=14)}',
                'token': token[0],
                'connection_string': connection_string
            }
        )

Executing: 100%|██████████| 22/22 [00:04<00:00,  4.94cell/s]


Executing: 100%|██████████| 22/22 [00:04<00:00,  5.13cell/s]


Executing: 100%|██████████| 22/22 [00:04<00:00,  5.09cell/s]


Executing: 100%|██████████| 22/22 [00:04<00:00,  5.31cell/s]


Executing: 100%|██████████| 22/22 [00:04<00:00,  5.13cell/s]


...

Executing: 100%|██████████| 22/22 [00:04<00:00,  5.05cell/s]


Executing: 100%|██████████| 22/22 [00:04<00:00,  5.03cell/s]


Executing: 100%|██████████| 22/22 [00:04<00:00,  5.01cell/s]


Executing: 100%|██████████| 22/22 [00:04<00:00,  5.06cell/s]


Executing: 100%|██████████| 22/22 [00:04<00:00,  4.99cell/s]


To check with BTC as an example of whether the above process gave desired results, I try the following few cells.

In [55]:
nb = sb.read_notebook('my_outputs/my_analysis_BTC_2021-12-01 00:00:00.ipynb')

In [62]:
nb.scraps['prices'].data.head()

,open,high,low,close,volume,volumeUSD,token,chain,return,volatility,max_drawdown
ts,,,,,,,,,,,
2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,BTC,-0.005834,NaN,0.000000
2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,BTC,0.010967,NaN,0.000000
2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,BTC,-0.010215,NaN,-0.010163
2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,BTC,0.004841,0.144076,-0.010163
2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,BTC,0.001681,0.105531,-0.010163


Now I read all notebook files and summarize the values requested by our homework instructions.

In [50]:
nbs = sb.read_notebooks('my_outputs') # read all notebooks

In [73]:
pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'mean_return': scrap['prices'].data['return'].mean(),
        'mean_volatility': scrap['prices'].data.volatility.mean(),
        'max_drawdown': scrap['prices'].data.max_drawdown.min(),
        'price_range': scrap['prices'].data.close[-1] - scrap['prices'].data.close[0]}) 
    for book, scrap in nbs.notebook_scraps.items()
]).set_index('token')

,mean_return,mean_volatility,max_drawdown,price_range
token,,,,
AAVE,-1.252962e-03,0.237465,-0.404689,-88.3180
ADA,-6.161726e-04,0.250735,-0.313680,-0.2849
ATOM,-7.509726e-04,0.338880,-0.358851,-5.8280
AVAX,-1.001603e-03,0.318885,-0.394209,-35.1400
BTC,-5.103292e-04,0.169846,-0.205844,-8698.9800
COMP,-1.238412e-03,0.255816,-0.378144,-93.6200
CRV,-1.077334e-03,0.362593,-0.394862,-1.6295
DOT,-1.149871e-03,0.276084,-0.348681,-11.8820
ETH,-5.734818e-04,0.196771,-0.218952,-794.4600
